In [1]:
import pandas as pd
import numpy as np
import math 
import tensorflow as tf
df = pd.read_csv("../../data/raw/news/newsapiorg_labelled.csv", index_col=0)
df.head()

,title,publishedAt,url,content,vader_neg,vader_neu,vader_pos,vader_comp,vader_label,label,include,relevant
Column1,,,,,,,,,,,,
1,Key remarks from Russian c.bank governor after...,2022-02-11T12:26:00Z,https://www.reuters.com/markets/currencies/key...,"MOSCOW, Feb 11 (Reuters) - The Russian Central...",0.000,1.000,0.0,0.0000,0,0,1,0
2,LIVE MARKETS Forex: worrying about an inverted...,2022-02-18T12:33:00Z,https://www.reuters.com/markets/stocks/live-ma...,Feb 18 - Welcome to the home for real-time cov...,0.211,0.789,0.0,-0.3400,-1,-1,1,1
3,Dollar takes a break from this week's slide ah...,2022-02-03T02:18:00Z,https://www.reuters.com/markets/europe/dollar-...,"SINGAPORE, Feb 3 (Reuters) - The dollar found ...",0.000,1.000,0.0,0.0000,0,-1,1,1
4,Euro bounce pauses ahead of U.S. inflation - R...,2022-02-08T01:16:00Z,https://www.reuters.com/business/euro-bounce-p...,"SINGAPORE, Feb 8 (Reuters) - A resurgent euro ...",0.000,1.000,0.0,0.0000,0,1,1,1
5,Dollar near two-week high amid jitters over ha...,2022-01-25T01:52:00Z,https://www.reuters.com/markets/europe/dollar-...,"TOKYO, Jan 25 (Reuters) - The safe-haven U.S. ...",0.197,0.803,0.0,-0.4019,-1,-1,1,1


In [2]:
df.label.value_counts()

 0    1049
-1     569
 1     535
Name: label, dtype: int64

In [3]:
df.loc[df["label"] == 0, 'label'] = 2
df.loc[df['label'] == 1, 'label'] = 0
df.loc[df['label'] == -1, 'label'] = 1
df.label.value_counts() 
#0: Positive
#1: Negative
#2: Neutral 

2    1049
1     569
0     535
Name: label, dtype: int64

In [4]:
import sklearn
from sklearn.model_selection import train_test_split
train_inputs, valid_inputs, train_labels, valid_labels = train_test_split(df.title, df.label, test_size = 0.2)

In [5]:
from datasets import Dataset
train_ds_raw = Dataset.from_dict({'text': train_inputs, 'labels': train_labels})
valid_ds_raw = Dataset.from_dict({'text': valid_inputs, 'labels': valid_labels})

In [6]:
train_ds_raw

Dataset({
    features: ['text', 'labels'],
    num_rows: 1722
})

In [7]:
valid_ds_raw

Dataset({
    features: ['text', 'labels'],
    num_rows: 431
})

In [8]:
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    AutoModelForSequenceClassification
)

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

In [9]:
def tokenize(examples):
    return tokenizer(examples['text'], truncation=True)

train_ds = train_ds_raw.map(tokenize, batched=True)
valid_ds = valid_ds_raw.map(tokenize, batched=True)


100%|██████████| 1/1 [00:00<00:00, 83.32ba/s]


In [10]:
train_ds

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1722
})

In [11]:
valid_ds

Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 431
})

In [12]:
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
MODEL_PATH = "../../results/models/FinBERT_v1.0"
train_args = TrainingArguments(
    MODEL_PATH,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    learning_rate=2e-5,
    weight_decay=0.01,
    warmup_ratio = 0.1,
    evaluation_strategy='epoch',
)

In [13]:
from sklearn.metrics import accuracy_score

def evaluation(eval_preds):
    logits, labels = eval_preds
    preds = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, preds)}

In [14]:
data_collator = DataCollatorWithPadding(tokenizer)
trainer = Trainer(
    model,
    train_args,
    train_dataset=train_ds,
    eval_dataset=valid_ds,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=evaluation,
)

In [15]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\wangt\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1722
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 540
 20%|██        | 108/540 [10:45<40:55,  5.68s/it] The following columns in the evalua

{'eval_loss': 0.5825806260108948, 'eval_accuracy': 0.7633410672853829, 'eval_runtime': 63.4456, 'eval_samples_per_second': 6.793, 'eval_steps_per_second': 0.426, 'epoch': 1.0}


 40%|████      | 216/540 [21:38<30:03,  5.57s/it]  The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 431
  Batch size = 16
                                                 
 40%|████      | 216/540 [22:33<30:03,  5.57s/it]

{'eval_loss': 0.5182616710662842, 'eval_accuracy': 0.814385150812065, 'eval_runtime': 54.8006, 'eval_samples_per_second': 7.865, 'eval_steps_per_second': 0.493, 'epoch': 2.0}


 60%|██████    | 324/540 [33:41<17:45,  4.93s/it]  The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 431
  Batch size = 16
                                                 
 60%|██████    | 324/540 [34:38<17:45,  4.93s/it]

...

 93%|█████████▎| 500/540 [52:21<03:51,  5.78s/it]Saving model checkpoint to ../../results/models/FinBERT_v1.0\checkpoint-500
Configuration saved in ../../results/models/FinBERT_v1.0\checkpoint-500\config.json


{'loss': 0.3865, 'learning_rate': 1.646090534979424e-06, 'epoch': 4.63}


Model weights saved in ../../results/models/FinBERT_v1.0\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ../../results/models/FinBERT_v1.0\checkpoint-500\tokenizer_config.json
Special tokens file saved in ../../results/models/FinBERT_v1.0\checkpoint-500\special_tokens_map.json
100%|██████████| 540/540 [55:49<00:00,  4.56s/it]The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 431
  Batch size = 16
                                                 
100%|██████████| 540/540 [56:29<00:00,  4.56s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 540/540 [56:29<00:00,  6.28s/it]

{'eval_loss': 0.6239088773727417, 'eval_accuracy': 0.8329466357308585, 'eval_runtime': 40.6166, 'eval_samples_per_second': 10.611, 'eval_steps_per_second': 0.665, 'epoch': 5.0}
{'train_runtime': 3389.8008, 'train_samples_per_second': 2.54, 'train_steps_per_second': 0.159, 'train_loss': 0.3734118797160961, 'epoch': 5.0}


TrainOutput(global_step=540, training_loss=0.3734118797160961, metrics={'train_runtime': 3389.8008, 'train_samples_per_second': 2.54, 'train_steps_per_second': 0.159, 'train_loss': 0.3734118797160961, 'epoch': 5.0})

In [16]:
trainer.save_model(MODEL_PATH)

Saving model checkpoint to ../../results/models/FinBERT_v1.0
Configuration saved in ../../results/models/FinBERT_v1.0\config.json
Model weights saved in ../../results/models/FinBERT_v1.0\pytorch_model.bin
tokenizer config file saved in ../../results/models/FinBERT_v1.0\tokenizer_config.json
Special tokens file saved in ../../results/models/FinBERT_v1.0\special_tokens_map.json


In [17]:
trained_bert = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)

loading configuration file ../../results/models/FinBERT_v1.0\config.json
Model config BertConfig {
  "_name_or_path": "../../results/models/FinBERT_v1.0",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "positive",
    "1": "negative",
    "2": "neutral"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative": 1,
    "neutral": 2,
    "positive": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.18.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

l

In [18]:
test_ds = valid_ds_raw.map(tokenize, batched=False)
test_ds

100%|██████████| 431/431 [00:00<00:00, 3918.19ex/s]


Dataset({
    features: ['text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 431
})

In [19]:
trained_model = Trainer(
    trained_bert,
    tokenizer=tokenizer,
)
output = trained_model.predict(
    test_dataset=test_ds
)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 431
  Batch size = 8
100%|██████████| 54/54 [00:31<00:00,  1.55it/s]

In [20]:
output

PredictionOutput(predictions=array([[-1.9047167 ,  3.4954553 , -1.1367399 ],
       [-1.5548592 , -1.685975  ,  3.67654   ],
       [-1.8579643 ,  3.4466624 , -1.3580679 ],
       ...,
       [-1.6760224 , -0.4588015 ,  2.7747884 ],
       [-2.2575953 ,  3.179277  ,  0.30187687],
       [-1.6353482 , -1.3618524 ,  3.5516517 ]], dtype=float32), label_ids=array([1, 2, 1, 0, 2, 0, 0, 0, 2, 2, 2, 1, 2, 2, 0, 1, 2, 2, 1, 2, 2, 1,
       2, 2, 2, 0, 2, 1, 2, 2, 2, 2, 0, 1, 1, 1, 2, 2, 1, 0, 2, 1, 2, 2,
       2, 2, 1, 2, 0, 0, 2, 1, 2, 2, 0, 0, 0, 2, 2, 0, 1, 2, 2, 2, 0, 2,
       0, 0, 2, 0, 0, 2, 0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2,
       2, 2, 0, 2, 2, 1, 2, 0, 2, 1, 2, 1, 2, 0, 2, 2, 2, 2, 1, 2, 2, 0,
       1, 0, 0, 0, 0, 1, 0, 2, 2, 0, 1, 1, 2, 2, 2, 2, 2, 1, 0, 2, 2, 0,
       1, 0, 2, 1, 1, 2, 0, 2, 0, 2, 2, 0, 2, 0, 2, 0, 1, 2, 2, 1, 0, 0,
       2, 1, 0, 1, 1, 1, 2, 1, 2, 0, 1, 0, 2, 0, 1, 1, 1, 1, 2, 2, 2, 0,
       0, 2, 0, 1, 2, 2, 0, 1, 0, 2, 2, 2, 2, 0, 2, 2, 1, 0,

In [21]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['label'])
encoder.inverse_transform([np.argmax(i) for i in output.predictions])
preds = [np.argmax(i) for i in output.predictions]

In [24]:
preds


[1,
 2,
 1,
 0,
 2,
 0,
 2,
 2,
 2,
 0,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 0,
 2,
 1,
 2,
 2,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 0,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 0,
 0,
 1,
 1,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 0,
 1,
 2,
 2,
 2,
 2,
 2,
 0,
 2,
 0,
 0,
 0,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 0,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 2,
 0,
 0,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 2,
 1,
 2,
 0,
 1,
 2,
 2,
 2,
 1,
 2,
 0,
 2,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 2,
 2,
 0,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 0,
 1,
 1,
 2,
 2,
 2,
 0,
 0,
 2,
 0,
 2,
 2,
 2,
 2,
 1,
 2,
 0,
 1,
 0,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 0,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 2,
 2,
 2,
 0,
 0,
 0,
 0,
 1,
 2,
 2,
 0,
 1,
 0,
 2,
 2,
 2,
 2,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 0,
 1,
 2,
 1,
 0,
 2,
 0,
 2,
 1,
 0,
 0,
 1,
 1,
 2,
 2,
 1,
 0,
 0,
 2,
 2,
 2,
 0,
 1,
 2,
 1,
 2,
 2,
 0,
 2,
 1,
 1,
 2,
 0,
 0,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 2,
 1,
 1,


In [25]:
accuracy_score(valid_labels, preds)

0.8329466357308585